<a href="https://colab.research.google.com/github/SakshiUK/BharatInternStockPricePrediction/blob/main/Copy_of_iner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, quote
import os
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import cmudict
import nltk
import re
import string

nltk.download('punkt')
nltk.download('cmudict')

def extract_article_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.text.strip() if soup.title else "Untitled"
        article_content = soup.find('div', class_='td-post-content tagdiv-type')
        if article_content:
            article_text = '\n'.join(p.text.strip() for p in article_content.find_all('p'))
        else:
            article_text = "Article content not found."

        return title, article_text
    except Exception as e:
        print(f"Error extracting content from {url}: {e}")
        return "Untitled", None


def save_to_text_file(file_name, title, article_text):
    with open(file_name, 'w', encoding='utf-8', errors='replace') as file:
        file.write(f"Title: {title}\n\n")
        file.write(article_text)

def read_stop_words(stop_word_files):
    stop_words = set()
    for file_name in stop_word_files:
        with open(file_name, 'r', encoding='iso-8859-1') as file:
            stop_words.update(word.strip() for word in file.readlines())
    return stop_words

def read_positive_negative_words(positive_file, negative_file):
    with open(positive_file, 'r', encoding='ISO-8859-1') as file:
        positive_words = [word.strip() for word in file.readlines()]

    with open(negative_file, 'r', encoding='ISO-8859-1') as file:
        negative_words = [word.strip() for word in file.readlines()]

    return positive_words, negative_words


def clean_text(text, stop_words):
    cleaned_text=' '.join(word.strip(string.punctuation) for word in text.split() if word.lower() not in stop_words)
    return cleaned_text

def calculate_positive_score(text,positive_words):
    tokens=word_tokenize(text)
    positive_score=sum(1 for token in tokens if token.lower() in positive_words)
    return positive_score

def calculate_negative_score(text,negative_words):
    tokens=word_tokenize(text)
    negative_score=sum(-1 for token in tokens if token.lower() in negative_words)
    return negative_score

def calculate_polarity_score(text,positive_words,negative_words):
    tokens=word_tokenize(text)
    negative_score=sum(-1 for token in tokens if token.lower() in negative_words)
    positive_score=sum(-1 for token in tokens if token.lower() in positive_words)
    polarity_score=(positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    return  polarity_score

def calculate_subjectivity_score(text,positive_words,negative_words):
    tokens=word_tokenize(text)
    total_words=len(tokens)
    negative_score=sum(-1 for token in tokens if token.lower() in negative_words)
    positive_score=sum(-1 for token in tokens if token.lower() in positive_words)
    subjectivity_score = (positive_score + negative_score)/ ((total_words) + 0.000001)
    return subjectivity_score

def calculate_complex_words(text):
    tokens = word_tokenize(text)
    pronouncing_dict = cmudict.dict()
    complex_words = [word for word in tokens if word.isalpha() and count_syllables(word) > 2]
    num_complex_words = len(complex_words)
    return num_complex_words

def calculate_average_word_length(text):
    tokens=word_tokenize(text)
    no_of_words=len(tokens)
    sum_of_chars=sum(len(token) for token in tokens)
    average_word_length=sum_of_chars/no_of_words
    return average_word_length

def calculate_sentences(text):
    tokens=sent_tokenize(text)
    no_of_sentences=len(tokens)
    return no_of_sentences

def calculate_avg_sent_length(text):
    no_of_words=len(word_tokenize(text))
    no_of_sentences=calculate_sentences(text)
    average_sentence_length=no_of_words/no_of_sentences
    return average_sentence_length

def calculate_words(text):
    tokens=word_tokenize(text)
    num_of_words=len(tokens)
    return num_of_words

def calculate_percentage_complex_words(text):
    num_complex_words=calculate_complex_words(text)
    no_of_words=calculate_words(text)
    percentage_complex_words=num_complex_words/no_of_words
    return percentage_complex_words

def calculate_fog_index(text):
    avg_sent_len=calculate_avg_sent_length(text)
    per_complex_words=calculate_percentage_complex_words(text)
    fog_index=0.4*(avg_sent_len + per_complex_words)
    return fog_index

def calculate_personal_pronouns(text):
    pronoun_pattern=re.compile(r'\b(?:I|we|my|ours|us)\b',flags=re.IGNORECASE)
    pronoun_matches=pronoun_pattern.findall(text)
    pronoun_count=len(pronoun_matches)
    return pronoun_count

def count_syllables(word):
    vowels ="aeiouy"
    word=word.lower()
    count=0
    prev_char=None

    for char in word:
        if char in vowels and (prev_char is None or prev_char not in vowels):
            count+=1
        prev_char=char

    if word.endswith("es") or word.endswith("ed"):
        count-=1

    return max(count,1)

def calculate_syllables(text):
    tokens=word_tokenize(text)
    syllables_count=sum(count_syllables(token) for token in tokens)
    return syllables_count


def main():

    df = pd.read_excel('Input.xlsx')
    newdf=pd.read_excel('Output Data Structure.xlsx')
    stop_word_files=['/content/StopWords_Auditor.txt','/content/StopWords_Currencies.txt','/content/StopWords_DatesandNumbers.txt','/content/StopWords_Generic.txt','/content/StopWords_GenericLong.txt','/content/StopWords_Geographic.txt','/content/StopWords_Names.txt']
    stop_words = read_stop_words(stop_word_files)
    positive_file = '/content/positive-words.txt'
    negative_file = '/content/StopWords_Names.txt'
    positive_words, negative_words = read_positive_negative_words(positive_file, negative_file)
    for index, row in df.iterrows():
        url = row['URL']
        url_id =  row['URL_ID']
        title, article_text = extract_article_text(url)

        if title and article_text:

            cleaned_article_text = clean_text(article_text, stop_words)

            filtered_positive_words = [word for word in positive_words if word.lower() not in stop_words]
            filtered_negative_words = [word for word in negative_words if word.lower() not in stop_words]

            positive_score = calculate_positive_score(cleaned_article_text, positive_words)
            negative_score = calculate_negative_score(cleaned_article_text, positive_words)
            polarity_score = calculate_polarity_score(cleaned_article_text, positive_words, negative_words)
            subjectivity_score = calculate_subjectivity_score(cleaned_article_text, positive_words, negative_words)

            average_word_length=calculate_average_word_length(cleaned_article_text)

            pronoun_count=calculate_personal_pronouns(cleaned_article_text)

            syllables_count=calculate_syllables(cleaned_article_text)

            word_count=len(word_tokenize(cleaned_article_text))

            num_complex_words=calculate_complex_words(cleaned_article_text)

            num_words_per_sent=word_count/num_complex_words

            average_sentence_length=calculate_avg_sent_length(cleaned_article_text)
            percentage_complex_words=calculate_percentage_complex_words(cleaned_article_text)
            fog_index=calculate_fog_index(cleaned_article_text)

            file_name = f"{url_id}_cleaned.txt"
            save_to_text_file(file_name, title, cleaned_article_text)

            existing_sheet = pd.read_excel('Output Data Structure.xlsx')

            # ['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
      #  'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
      #  'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
      #  'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
      #  'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']

            existing_sheet.at[index, 'POSITIVE SCORE'] = positive_score
            existing_sheet.at[index, 'NEGATIVE SCORE'] = negative_score
            existing_sheet.at[index, 'POLARITY SCORE'] = polarity_score
            existing_sheet.at[index, 'SUBJECTIVITY SCORE'] = subjectivity_score
            existing_sheet.at[index, 'AVG SENTENCE LENGTH'] = average_sentence_length
            existing_sheet.at[index, 'PERCENTAGE OF COMPLEX WORDS'] = percentage_complex_words
            existing_sheet.at[index, 'FOG INDEX'] = fog_index
            existing_sheet.at[index, 'AVG NUMBER OF WORDS PER SENTENCE'] = num_words_per_sent
            existing_sheet.at[index, 'COMPLEX WORD COUNT'] = num_complex_words
            existing_sheet.at[index, 'WORD COUNT'] = word_count
            existing_sheet.at[index, 'SYLLABLE PER WORD'] = syllables_count
            existing_sheet.at[index, 'PERSONAL PRONOUNS'] = pronoun_count
            existing_sheet.at[index, 'AVG WORD LENGTH'] = average_word_length

            existing_sheet.to_excel('Output Data Structure.xlsx', index=False)


if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
